In [ ]:
# Processing Brian's kw data (data already extracted from MeterDataTest folder)
# Complete this before second part of this notebook

In [ ]:
import pandas as pd

In [ ]:
import os, sys
sys.path.append(os.path.abspath('..'))
from processing_kw import load_data, process_kw_data

In [ ]:
# data input csv
data_path = 'save_data_col.csv'

# guide of the meters with their meter model types
info_path = 'all_headers.csv'

In [ ]:
# load meter data and info from csvs into dataframes, cleaned and reformatted
df, info_df = load_data(data_path, info_path)

In [ ]:
# process data for only dates sept 7-9
#df = df[df['datetime'].dt.date.isin([
#    pd.to_datetime('2025-09-07').date(),
#    pd.to_datetime('2025-09-08').date(),
#    pd.to_datetime('2025-09-09').date()])]

In [ ]:
# processed kw data in dataframe
result_df = process_kw_data(df, info_df)

In [ ]:
# create csv of processed data
result_df.to_csv('processed_kw.csv', index=False)

In [ ]:
# Compare processed brians kw ('mean_kw') data to auroras kw ('mean')

In [1]:
import os, sys
sys.path.append(os.path.abspath('..'))
from processing_kw import load_data_for_comparison, create_plots_pdf, get_comparison_info

In [2]:
# brian input csv
#brian_csv = 'sept07-09_kw.csv'
brian_csv = 'processed_kw.csv'

# aurora input csv
#aurora_csv = 'blue_pillar_data.csv'
aurora_csv = 'aurora_processed_kw.csv'

# create merged dataframe of the input csvs and get the list of meters
merged_df, meters = load_data_for_comparison(brian_csv, aurora_csv)

# optional create csv of merged dataframe
#merged_df.to_csv('merged_brian_aurora.csv', ignore_index=True)

OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 0001-01-01 05:30:00, at position 152935. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
# create pdf of plots comparing brian vs aurora kw data per meter
create_plots_pdf(merged_df, meters, 'aurora_vs_curr.pdf')

In [ ]:
# create dataframe of info summarizing comparison brian vs aurora kw data per meter
info_df = get_comparison_info(merged_df, meters)

# create csv of comparison info dataframe
info_df.to_csv('brian_bp_comparison_info.csv', index=True)